In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
df = pd.read_csv("Bank_Personal_Loan_Modelling.csv")
df.drop(columns=['ID','ZIP Code'], inplace=True)
y = df.pop('Personal Loan')
x = df
x = StandardScaler().fit_transform(x)

In [ ]:
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.25)

In [ ]:
def fitness_function(individual):
    model.set_weights(individual)

    _, accuracy = model.evaluate(x_ts, y_ts, verbose=0)

    return accuracy

POPULATION_SIZE = 20
NUM_GENERATIONS = 50
MUTATION_RATE = 0.1
SELECTION_CRITERIA = 'roulette wheel'


def roulette_wheel_selection(fitness_scores, num_parents):
    fitness_sum = np.sum(fitness_scores)
    fitness_probs = fitness_scores / fitness_sum
    parents_indices = np.random.choice(range(len(fitness_probs)), size=num_parents, replace=False, p=fitness_probs)
    return parents_indices


def crossover(parent1, parent2):

    offspring = np.empty_like(parent1)

    for index in np.ndindex(parent1.shape):
        if np.random.rand() < 0.5:
            offspring[index] = parent1[index]
        else:
            offspring[index] = parent2[index]

    return offspring



def mutate(individual, mutation_rate):
    mutated_individual = []
    for i in range(len(individual)):
        if np.random.uniform() < mutation_rate:
            mutated_gene = np.random.normal(size=individual[i].shape)
        else:
            mutated_gene = individual[i]
        mutated_individual.append(mutated_gene)
    return mutated_individual

model = Sequential()
model.add(tf.keras.Input(shape=(11,)))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

population = []
for i in range(POPULATION_SIZE):
    weights = model.get_weights()
    for j in range(len(weights)):
        weights[j] = np.random.normal(size=weights[j].shape)
    population.append(weights)

def create_individual():
    return individual


for generation in range(NUM_GENERATIONS):
    fitness_scores = [fitness_function(individual) for individual in population]

    sorted_indices = np.argsort(fitness_scores)[::-1]

    if sorted_indices.size > 0:
        population = [population[index] for index in sorted_indices[:POPULATION_SIZE]]
    else:
        population = [create_individual() for _ in range(POPULATION_SIZE)]

    new_population = []
    for _ in range(POPULATION_SIZE):
        parent1, parent2 = np.random.choice(population, size=2, replace=False)

        child = []
        for gene_index in range(len(parent1)):
            if np.random.random() < 0.5:
                child.append(parent1[gene_index])
            else:
                child.append(parent2[gene_index])

        for gene_index in range(len(child)):
            if np.random.random() < MUTATION_RATE:
                child[gene_index] = mutate(child[gene_index])

        new_population.append(child)

    population = new_population

In [ ]:
model.set_weights(population[0])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(tf.keras.Input(shape=(11,)))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.fit(x=x_tr, y=y_tr, verbose=1, epochs=50)

In [ ]:
# Accuracy using Genetic Algorithm
accuracy = model.evaluate(x=x_ts, y=y_ts, verbose=0)[1]
print(f"Test accuracy: {accuracy*100:.4f}%")